In [ ]:
from datetime import datetime
from sklearn.metrics import roc_auc_score, plot_roc_curve

import json
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 1000)
from configparser import ConfigParser

import mysql.connector
import seaborn as sns
from datetime import date
from datetime import datetime
from datetime import timedelta

from getpass import getpass
import os
import glob
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim

import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from itertools import cycle
from tqdm.notebook import tqdm
import re
from sklearn.metrics import roc_auc_score, plot_roc_curve, roc_curve

import matplotlib.pyplot as plt
import seaborn
from collections import defaultdict
import shap

from sklearn import svm
from sklearn.neighbors import KDTree
import sqlalchemy
from sqlalchemy.pool import NullPool

In [ ]:
## server config

In [ ]:
def make_location(addr1, addr2, addr3, addr4):
    if type(addr1) != str: addr1 = ''
    if type(addr2) != str: addr2 = ''
    if type(addr3) != str: addr3 = ''
    if type(addr4) != str: addr4 = ''
        
    return f'{addr1} {addr2} {addr3} {addr4}'

In [ ]:
class ReverGeoCoder:
    def __init__(self, src_file='./location_korea_20210624.csv'):
        df_location = pd.read_csv(src_file)
        df_location = df_location[df_location['address_1'].notnull()]
        df_location = df_location[['_id','loc_1','loc_2','address_1','address_2','address_3','address_4','address_5']]
        df_location = df_location.rename(columns={'_id':'location_id'})
        df_location = df_location.reset_index(drop=True)
        df_location['loc_1'] = df_location['loc_1'].astype(float)
        df_location['loc_2'] = df_location['loc_2'].astype(float)

        print("Model Source loaded:", df_location.shape)
        self.kdt = KDTree(df_location[['loc_1', 'loc_2']], leaf_size=30, metric='euclidean')
        self.df_location = df_location.copy()


    def query_one(self, lati, longi):
        ret = self.kdt.query([[lati, longi]], k=1, return_distance=False)
        print("matching index:", ret)  # Print index
        return self.df_location.loc[[x[0] for x in ret], :]
    
    def query_list(self, list_location):
        """list should have [lat, lng] as elements. Ex) [[35.18692, 126.8927], [37.69954, 127.1940], [37.26776, 126.9961]]"""
        ret = self.kdt.query(list_location, k=1, return_distance=False)
        print("matching index:", ret)  # Print index
        return self.df_location.loc[[x[0] for x in ret], :]
    
    
    def query_df(self, df_query):
        """df should have columns ['lat', 'lng']"""
        ret = self.kdt.query(df_query[['lat', 'lng']].copy(), k=1, return_distance=False)
        df_ret = self.df_location.loc[[x[0] for x in ret], :]
        df_ret['loca'] = (df_ret.apply(lambda x: 
                                           make_location(x['address_1'], x['address_2'],
                                                         x['address_3'], x['address_4']), axis=1))
        df_query = df_query.reset_index(drop=True)
        df_ret = df_ret.reset_index(drop=True)
        df_query['location'] = df_ret['loca']
        return df_query
        
coder = ReverGeoCoder()

display(coder.query_one(35.18692, 126.8927))
display(coder.query_list([[35.18692, 126.8927], [37.69954, 127.1940], [37.26776, 126.9961]]))


In [ ]:
def get_postgresql_engine(path_to_yml='./secret.ini'):
    config = ConfigParser()
    config.read(path_to_yml)

    HOSTNAME = config["kt_db"]["HOSTNAME"]
    PORT = int(config["kt_db"]["PORT"])
    USERNAME = config["kt_db"]["USERNAME"]
    PASSWORD = config["kt_db"]["PASSWORD"]
    DATABASE = config["kt_db"]["DATABASE"]
    
    url = f"postgresql://{USERNAME}:{PASSWORD}@{HOSTNAME}:{PORT}/{DATABASE}"
    return sqlalchemy.create_engine(url, client_encoding='utf8', poolclass=NullPool)

In [ ]:
kt_engine = get_postgresql_engine()

In [ ]:
qry = "SELECT * FROM shine2_general_pcr"
df_pcr = pd.read_sql(qry, kt_engine)

In [ ]:
df_pcr = df_pcr[df_pcr['lat'].notnull()]
df_pcr = df_pcr[df_pcr['lng'].notnull()]
df_pcr['lat'] = df_pcr['lat'].astype(float)
df_pcr['lng'] = df_pcr['lng'].astype(float)
print(df_pcr.shape)

df_pcr = df_pcr[(df_pcr['lat'] >= 33) & 
              (df_pcr['lat'] <= 39) &
              (df_pcr['lng'] >= 124) &
              (df_pcr['lng'] <= 132)]
print(df_pcr.shape)
df_pcr.head()

In [ ]:
df_pcr_loc = coder.query_df(df_pcr)
df_pcr_loc.head()

### md_data에 location 필드 넣어서, 당시 gps 해당 주소 넣어준 csv 불러옴

In [ ]:
# df_md_data_location = pd.read_csv('md_data_with_location.csv')
# df_md_data_location = pd.read_csv('md_data_with_location_20221102.csv')

In [ ]:
# df_md_data_location.shape[0]

In [ ]:
# df_md_data_location.head()

In [ ]:
# df_md_data_location

In [ ]:
# df_md_data_location.shape[0]

### 샤인데이터에 들어있는 시군구 리스트 이름 뽑아와서 주소를 시군구로 바꿔주는 작업

In [ ]:
df_sigungu_name = pd.read_csv('sigungu_name.csv')

In [ ]:
df_sigungu_name

In [ ]:
sigungu_dict = {}
for addr_1, addr_2 in df_sigungu_name.values:
    if addr_1 not in sigungu_dict:
        sigungu_dict[addr_1] = []
    sigungu_dict[addr_1].append(addr_2)
sigungu_dict

In [ ]:
def apply_and_concat(df, field, func, cols_name):
    return pd.concat((df, df[field].apply(lambda cell: pd.Series(func(cell), index=cols_name))), axis=1)

In [ ]:
def make_sidosigungu(location):
    sido = ''
    sigungu = ''
    if type(location) == float:
        return sido, sigungu
    for addr1 in sigungu_dict:
        if addr1 in location:
            sido = addr1
            for addr2 in sigungu_dict[addr1]:
                if addr2 in location:
                    sigungu = addr2
    return sido, sigungu

In [ ]:
cols_name = ['sido', 'sigungu']
df_pcr_loc = apply_and_concat(df_pcr_loc, 'location', make_sidosigungu, cols_name)

In [ ]:
df_pcr_loc.head()

### 이벤트 기간
- 1차: 2.28~4.4
- 2차: 8.16~10.30

### sido, sigungu 존재하는 데이터 갯수 체크

In [ ]:
df_pcr_loc[(df_pcr_loc['sido'] == '') |  (df_pcr_loc['sigungu'] == '')].shape[0]

In [ ]:
df_pcr_loc[~((df_pcr_loc['sido'] == '') |  (df_pcr_loc['sigungu'] == ''))].shape[0]

In [ ]:
df_pcr_loc['sido'] = df_pcr_loc['sido'].fillna('')
df_pcr_loc['sigungu'] = df_pcr_loc['sigungu'].fillna('')

### 여기서 일단 sigungu 있는 애들만 따로 저장

In [ ]:
df_pcr_loc = (df_pcr_loc[~((df_pcr_loc['sido'] == '') | (df_pcr_loc['sigungu'] == ''))])

In [ ]:
df_pcr_loc.head()

In [ ]:
df_pcr_loc[df_pcr_loc['pcr_result'] == '1'].shape[0], df_pcr_loc[df_pcr_loc['pcr_result'] == '2'].shape[0]

### baby 정보 가져오기

In [ ]:
df_kt_baby = pd.read_sql('SELECT * FROM shine2_general_user', kt_engine)

In [ ]:
df_kt_baby.columns

In [ ]:
df_kt_baby = df_kt_baby[~df_kt_baby['gender'].isnull()]

In [ ]:
df_kt_baby['birthday'] = df_kt_baby['birthday'].astype(str)

In [ ]:
df_kt_baby['birthday'].apply(lambda x : x[:4]).unique()

In [ ]:
df_kt_baby['gender'] = df_kt_baby['gender'].astype(int)

In [ ]:
def get_age(birthday):
    return datetime.now().year - int(birthday[:4])

In [ ]:
df_kt_baby['age'] = df_kt_baby['birthday'].apply(get_age)

In [ ]:
df_kt_baby['age_60_and_above'] = df_kt_baby['age'].apply(lambda age: 1 if age >= 60 else 0)

In [ ]:
df_kt_baby[df_kt_baby['age_60_and_above'] == 1]

### 시군구 넣어준 md_data에서 pcr 데이터랑 selfcheck 데이터 따로 뽑기

In [ ]:
patients_str = ','.join(df_pcr_loc['patient_id'].astype(str).unique().tolist())

In [ ]:
df_selfcheck = pd.read_sql(f"SELECT * FROM shine2_general_selfcheck WHERE patient_id IN ({patients_str})", kt_engine)

In [ ]:
df_selfcheck['reg_date'].astype(str)

In [ ]:
df_pcr_loc.shape[0]

In [ ]:
df_pcr_loc[df_pcr_loc['pcr_result'] == '2'].shape[0], df_pcr_loc[df_pcr_loc['pcr_result'] == '1'].shape[0]

In [ ]:
df_pcr_loc['patient_id'].shape[0], df_pcr_loc['patient_id'].unique().shape[0]

In [ ]:
df_pcr_loc.shape[0]

In [ ]:
df_kt_baby['patient_id'] = df_kt_baby['patient_id'].astype(int)

### pcr 데이터에 유저 정보(생일, 성별, 나이, 60세이상) 입력 + pcr에 붙은 sido, sigungu 살림

In [ ]:
df_pcr_loc = pd.merge(df_pcr_loc, df_kt_baby, how='left', on='patient_id')

In [ ]:
df_pcr_loc.columns

In [ ]:
df_pcr_loc['pcr_date'] = df_pcr_loc['pcr_date'].astype(str)
df_selfcheck['selfcheck_date'] = df_selfcheck['selfcheck_date'].astype(str)

### pcr 데이터 입력 당시 시군구를 넣음

In [ ]:
df_res_temp = pd.merge(df_pcr_loc, df_selfcheck, how='inner', on='patient_id', suffixes=('_pcr', '_selfcheck'))

In [ ]:
df_res_temp.columns

In [ ]:
def generate_diff_date(selfcheck_date, pcr_date):
    selfcheck_date = pd.to_datetime(selfcheck_date, format='%Y-%m-%d')
    pcr_date = pd.to_datetime(pcr_date, format='%Y-%m-%d')
    delta = abs(int((selfcheck_date - pcr_date) / pd.Timedelta(days=1)))
    return delta

In [ ]:
df_res_temp['time_diff'] = df_res_temp.apply(lambda x: generate_diff_date(x['selfcheck_date'], x['pcr_date']), axis=1)

In [ ]:
df_res_temp = df_res_temp.sort_values('time_diff')
df_res_temp = df_res_temp.drop_duplicates(subset='_id_pcr', keep="first")

In [ ]:
df_res_temp = df_res_temp[df_res_temp['time_diff'] <= 14]

In [ ]:
df_res_temp.head()

In [ ]:
df_res = df_res_temp[['patient_id', 'gender', 'age', 'birthday', 'age_60_and_above',\
                                   'fever', 'sx_name', 'etc_status', 'is_abroad', 'is_visit', \
                                   'is_contact', 'is_isolation', 'selfcheck_reason',\
                                   'selfcheck_date', 'pcr_date', 'pcr_place', 'sido', 'sigungu', 'pcr_result']]

### 아래 방식대로하면 3시간 걸림...

In [ ]:

# df_res = pd.DataFrame([], columns=['baby_id', 'gender', 'age', 'birthday', 'age_60_and_above',\
#                                    'fever', 'sx_name', 'etc_status', 'is_abroad', 'is_visit', \
#                                    'is_contact', 'is_isolation', 'selfcheck_reason',\
#                                    'self_check_date', 'test_date', 'pcr_place', 'sido', 'sigungu', 'pcr_result'])

# display(df_res)
# cnt = 0

# is_confirmed = [0] * 3

# # self_check_num = {
# #     'old': [],
# #     'new': []
# # }

# self_check_cnt = defaultdict(int)
# for idx, (i, row) in enumerate(df_pcr_loc.iterrows()):
#     if idx % 1000 == 0: print(idx)
#     if row['pcr_date'] <= '2021-12-01': continue
#     pcr_date = pd.to_datetime(row['pcr_date'],format='%Y-%m-%d')
#     pcr_result = row['pcr_result']
#     lower_date = (pd.to_datetime(row['pcr_date'],format='%Y-%m-%d') - timedelta(days=14)).strftime("%Y-%m-%d")
#     upper_date = (pd.to_datetime(row['pcr_date'],format='%Y-%m-%d') + timedelta(days=14)).strftime("%Y-%m-%d")
#     df_self = df_selfcheck[df_selfcheck['patient_id'] == row['patient_id']]
    
#     df_self = df_self[(df_self['selfcheck_date'] >= lower_date) & (df_self['selfcheck_date'] <= upper_date)]
#     df_self = df_self.sort_values('reg_date')
# #     self_check_num['old'].append(df_self.shape[0])
# #     df_self = df_self.drop_duplicates(subset=['reg_date'])
# #     self_check_num['new'].append(df_self.shape[0])
#     self_check_cnt[df_self.shape[0]] += 1
#     if df_self.shape[0] == 0: continue
#     for i, ser_self in df_self.iterrows():
#         is_confirmed[int(row['pcr_result'])] += 1
#         append_row = ([row['patient_id'], row['gender'], row['age'], row['birthday'], 
#                        row['age_60_and_above'], ser_self['fever'], ser_self['sx_name'], ser_self['etc_status'],
#                        ser_self['is_abroad'], ser_self['is_visit'], ser_self['is_contact'], 
#                        ser_self['is_isolation'], ser_self['selfcheck_reason'], ser_self['selfcheck_date'], 
#                        row['pcr_date'], row['pcr_place'], row['sido'], row['sigungu'], row['pcr_result']])
#         df_res = df_res.append(pd.Series(append_row, index=df_res.columns), ignore_index=True)

In [ ]:
sx_name_dicts = {'마른 기침': '104',
 '피가 섞인 기침': '108',
 '가래': '107',
 '가슴통증': '102',
 '콧물': '106',
 '인후통(목통증)': '105',
 '숨참': '101',
 '근육통': '303',
 '두통': '302',
 '오한': '301',
 '미각소실': '309',
 '후각소실': '310',
 '늘어지고\n피곤함': '311',
 '입맛없음': '201',
 '설사': '204',
 '변비': '206',
 '눈 충혈': '312',
 '입술 파래짐': '103',
 '재채기': '109',
 '몸통발진': '313',
 '입술 주변 물집': '314',
 '기타': '999'}

In [ ]:
def symp_filter(sx_name, _other, is_contact, selfcheck_reason):
    symptoms = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    purposes = [1, 0, 0] # 'indication_other', 'indication_abroad', 'indication_contact'
    if sx_name != None and sx_name != '':
        symptom_check_list = [sx_name_dicts[i] for i in sx_name.split('`')]
        if '104' or '108' in symptom_check_list: symptoms[0] = 1
        if '105' in symptom_check_list: symptoms[1] = 1 # 가래, 인후통 따로
        if '101' in symptom_check_list: symptoms[2] = 1 # 가슴통증따로
        if '302' in symptom_check_list: symptoms[3] = 1
        #### 여기부터 기존에 없는 증상
        if '106' in symptom_check_list: symptoms[4] = 1
        if '303' in symptom_check_list: symptoms[5] = 1
        if '301' in symptom_check_list: symptoms[6] = 1
        if '309' in symptom_check_list: symptoms[7] = 1
        if '310' in symptom_check_list: symptoms[8] = 1
        if '107' in symptom_check_list: symptoms[9] = 1
        if '102' in symptom_check_list: symptoms[10] = 1
    
    if _other != None:
        if '두통' or '머리' in _other: symptoms[3] = 1
        if '인후' or '침' or '목' in _other: symptoms[1] = 1
        if '가래' in _other: symptoms[9] = 1
        if '기침' in _other: symptoms[0] = 1
    
    if is_contact == 1: purposes = [0, 0, 1]
    if selfcheck_reason == '6' and purposes[0] == 1: purposes = [0, 1, 0]
    return symptoms + purposes

In [ ]:
def apply_and_concat(df, sx_name, _other, is_contact, selfcheck_reason, func, cols_name):
    return pd.concat((df, df.apply(lambda x: pd.Series(func(x[sx_name], x[_other], \
                                   x[is_contact], x[selfcheck_reason]), index=cols_name), axis=1)), axis=1)

In [ ]:
df_res.columns

In [ ]:
df_res['fever'] = df_res['fever'].apply(lambda x: 0 if x == '' else x)

In [ ]:
cols = ['cough', 'sore_throat', 'shortness_of_breath',\
        'head_ache', 'runny_nose', 'muscle_pain', 'chills',\
        'loss_of_taste', 'loss_of_smell', 'sputum', 'chest_pain',\
        'indication_other', 'indication_abroad', 'indication_contact']

df_res = apply_and_concat(df_res, 'sx_name', 'etc_status', 'is_contact', 'selfcheck_reason', symp_filter, cols)

In [ ]:
df_res.columns

In [ ]:
df_res['weekday'] = pd.to_datetime(df_res['pcr_date']).dt.dayofweek

In [ ]:
df_res['pcr_result'] = df_res['pcr_result'].astype(int)

In [ ]:
df_res['age_ratio'] = df_res['age'] / 100

In [ ]:
df_res[df_res['pcr_result'] == 1].shape[0], df_res[df_res['pcr_result'] == 2].shape[0]

In [ ]:
df_res

### 가입일 데이터 제외 

In [ ]:
# qry = 'SELECT b._id as baby_id, u.join_date FROM md_user u JOIN md_baby b ON u._id = b.user_sn;'
# cols = ['baby_id', 'join_date']

# df_joindate = create_df(qry, cols, shine_conf)
# df_joindate['baby_id'] = df_joindate['baby_id'].astype(int)
# df_res['baby_id'] = df_res['baby_id'].astype(int)

# df_res = pd.merge(df_res, df_joindate, how='inner', on='baby_id')

In [ ]:
# df_res

In [ ]:
# df_res['join_date'] = df_res['join_date'].apply(lambda x: x[:10])

In [ ]:
df_res = df_res.sort_values('pcr_date', ascending=True)

In [ ]:
df_res

In [ ]:
df_res[df_res['pcr_result'] == 1].shape[0], df_res[df_res['pcr_result'] == 2].shape[0]

In [ ]:
df_res.columns # 증상과 요일이 채워짐

### 백신 정보인데 아직 인풋으로 안넣어줌

In [ ]:
# df_vac = df_md_data_location[df_md_data_location['type'] == 104][['_id', 'baby_id', 'data_1', 'data_2', 'data_3', 'data_7']]
# df_vac.columns = ['_id', 'baby_id', 'vac_date', 'vac_type', 'vac_degree', 'vac_confirmed']
# df_vac = df_vac[df_vac['vac_confirmed'] == 'Y']
# df_vac

In [ ]:
# def vac_degree_filter(baby_id, self_check_date):
#     vac_degree = 0
#     df_vac_info = df_vac[(df_vac['baby_id'] == str(baby_id)) & (df_vac['vac_date'] < self_check_date)]
#     df_vac_info = df_vac_info.sort_values('vac_degree')
#     if df_vac_info.shape[0] != 0:
#         vac_degree = int(df_vac_info.iloc[-1]['vac_degree'])
#     return vac_degree

In [ ]:
# df_res['vac_degree'] = df_res.apply(lambda df: vac_degree_filter(df['baby_id'], df['self_check_date']), axis=1)

In [ ]:
df_res['pcr_result'] = df_res['pcr_result'].astype(int)

In [ ]:
df_res['age_ratio'] = df_res['age'] / 100

In [ ]:
df_res[df_res['pcr_result'] == 1].shape[0], df_res[df_res['pcr_result'] == 2].shape[0]

In [ ]:
df_res.columns

In [ ]:
df_covid = df_res

### pcr 결과를 0: 음성 ,1: 양성으로 바꿔주는 작업

In [ ]:
df_covid['pcr_result'] = df_covid['pcr_result'].apply(lambda x: int(x) - 1)

### 본격적으로 유행도 붙여주는 작업

In [ ]:
def filter_date(date):
    year = f"20{date[-2:]}"
    month = date[:date.index('/')]
    month = f'0{month}' if len(month) == 1 else month
    day = date[date.index('/') + 1 : date.rindex('/')]
    day = f'0{day}' if len(day) == 1 else day
    res = f'{year}-{month}-{day}'
    return res

In [ ]:
def get_global_confirmed_df(df_timeseries_confirm):
    df_confirmed_global = df_timeseries_confirm.sum().reset_index()
    df_confirmed_global = df_confirmed_global.rename(columns={'index': 'global_confirmed_date', 0: 'global_confirmed_numsum'}).iloc[3:, :]
    df_confirmed_global = df_confirmed_global.reset_index(drop=True)
    df_confirmed_global['global_confirmed_num'] = df_confirmed_global['global_confirmed_numsum'] - df_confirmed_global.shift(1)['global_confirmed_numsum']
    df_confirmed_global = df_confirmed_global.dropna()
    df_confirmed_global['global_confirmed_num'] = df_confirmed_global['global_confirmed_num'].astype(int)
    df_confirmed_global['global_confirmed_date'] = df_confirmed_global['global_confirmed_date'].apply(filter_date)
    return df_confirmed_global

In [ ]:
def get_kor_confirmed_df(df_timeseries_confirm):
    rows = []
    for idx, values in enumerate(df_timeseries_confirm[df_timeseries_confirm['Country/Region'] == 'Korea, South'].iteritems()):
        if idx < 4: continue
        year = f"20{values[0][-2:]}"
        month = values[0][:values[0].index('/')]
        month = f'0{month}' if len(month) == 1 else month
        day = values[0][values[0].index('/') + 1 : values[0].rindex('/')]
        day = f'0{day}' if len(day) == 1 else day
        date = f'{year}-{month}-{day}'
        confirmed_num = values[1].item()
        rows.append([date, confirmed_num])
    df_numsum_confirmed = pd.DataFrame(rows, columns=['confirmed_date', 'confirmed_numsum'])
    df_numsum_confirmed['confirmed_num'] = df_numsum_confirmed['confirmed_numsum'] - df_numsum_confirmed.shift(1)['confirmed_numsum']
    df_numsum_confirmed = df_numsum_confirmed.dropna()
    df_numsum_confirmed['confirmed_num'] = df_numsum_confirmed['confirmed_num'].astype(int)
    return df_numsum_confirmed

In [ ]:
def get_confirmed_df():
    url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
    df_timeseries_confirm = pd.read_csv(url)
    display(df_timeseries_confirm)
    global_confirmed_df = get_global_confirmed_df(df_timeseries_confirm)
    kor_confirmed_df = get_kor_confirmed_df(df_timeseries_confirm)
    return global_confirmed_df, kor_confirmed_df

In [ ]:
def display_performance(clf, pred, valid_input, valid_gt):
    pred = np.array(pred)

    print('accuracy %.4f' % (accuracy_score(pred, valid_gt)))
    print('precision %.4f' % (precision_score(pred, valid_gt)))
    print('recall %.4f' % (recall_score(pred, valid_gt)))
    print('f1-score %.4f' % (f1_score(pred, valid_gt)))
    print(confusion_matrix(pred, valid_gt))
    plot_roc_curve(clf, valid_input, valid_gt)

In [ ]:
global_confirmed_df, kor_confirmed_df = get_confirmed_df()

In [ ]:
def kor_find_week_confirmed_num(confirmed_date):
    date_column='confirmed_date'
    cur_datetime = pd.to_datetime(confirmed_date, format='%Y-%m-%d')
    before_week_datetime = cur_datetime - timedelta(days=7)
    before_week_date = before_week_datetime.strftime('%Y-%m-%d')
    df = kor_confirmed_df[(before_week_date < kor_confirmed_df[date_column]) & (kor_confirmed_df[date_column] <= confirmed_date)]
    return int(df['confirmed_num'].mean() // 1)

In [ ]:
def global_find_week_confirmed_num(confirmed_date):
    date_column = 'global_confirmed_date'
    cur_datetime = pd.to_datetime(confirmed_date, format='%Y-%m-%d')
    before_week_datetime = cur_datetime - timedelta(days=7)
    before_week_date = before_week_datetime.strftime('%Y-%m-%d')
    df = global_confirmed_df[(before_week_date < global_confirmed_df[date_column]) & (global_confirmed_df[date_column] <= confirmed_date)]
    return int(df['global_confirmed_num'].mean() // 1)

In [ ]:
global_confirmed_df['global_confirmed_week_num'] = global_confirmed_df['global_confirmed_date'].apply(global_find_week_confirmed_num)

In [ ]:
kor_confirmed_df['confirmed_week_num'] = kor_confirmed_df['confirmed_date'].apply(kor_find_week_confirmed_num)

In [ ]:
kor_confirmed_df.tail()

In [ ]:
global_confirmed_df.tail()

In [ ]:
CONFIRMED_MAX = 11934 / 9.217 * 51.78  # 66033
GLOBAL_CONFIRMED_MAX = 1498041

In [ ]:
kor_confirmed_df['confirmed_ratio'] = kor_confirmed_df['confirmed_num'] / CONFIRMED_MAX
global_confirmed_df['global_confirmed_ratio'] = global_confirmed_df['global_confirmed_num'] / GLOBAL_CONFIRMED_MAX
kor_confirmed_df['confirmed_week_ratio'] = kor_confirmed_df['confirmed_week_num'] / CONFIRMED_MAX
global_confirmed_df['global_confirmed_week_ratio'] = global_confirmed_df['global_confirmed_week_num'] / GLOBAL_CONFIRMED_MAX

In [ ]:
df_pcr_loc

In [ ]:
df_sigungu_population = df_pcr_loc[['sido', 'sigungu']].drop_duplicates().reset_index(drop=True)

In [ ]:
df_sigungu_population

### 인구수로 nomalization
- 2022년 6월 기준 행정구역 총 인구수 
- https://jumin.mois.go.kr/ 여기 참고

In [ ]:
df_sigungu_population_raw = pd.read_csv('./sigungu_population/population_1.csv', encoding='cp949')
for i in range(2, 18):
    df_append = pd.read_csv(f'./sigungu_population/population_{i}.csv', encoding='cp949')
    df_sigungu_population_raw = pd.concat((df_sigungu_population_raw, df_append))
df_sigungu_population_raw.reset_index(drop=True)
df_sigungu_population_raw.head()

In [ ]:
for idx, row in df_sigungu_population.iterrows():
    sido = row['sido']
    sigungu = row['sigungu']
    if sido == '충북':
        sido = '충청북도'
    elif sido == '충남':
        sido = '충청남도'
    elif sido == '경북':
        sido = '경상북도'
    elif sido == '경남':
        sido = '경상남도'
    elif sido == '전북':
        sido = '전라북도'
    elif sido == '전남':
         sido = '전라남도'
    condit1 = df_sigungu_population_raw['행정구역'].str.contains(sido)
    condit2 = df_sigungu_population_raw['행정구역'].str.contains(sigungu)
    print(row['sido'], row['sigungu'])
    df_res = df_sigungu_population_raw[condit1 & condit2]
    
    if df_res.shape[0] == 0:
        print(0)
    elif df_res.shape[0] == 1:
        print(df_res['2022년06월_총인구수'].item().replace(',', ''))
    else:
        display(df_res)
        if sido == '인천' and sigungu == '동구':
            print(df_res[df_res['행정구역'] == '인천광역시 동구 (2814000000)']['2022년06월_총인구수'].item().replace(',', ''))
        elif sido == '경기' and sigungu == '양주':
            print(df_res[df_res['행정구역'] == '경기도 양주시 (4163000000)']['2022년06월_총인구수'].item().replace(',', ''))
        elif sido == '부산' and sigungu == '서구':
            print(df_res[df_res['행정구역'] == '부산광역시 서구 (2614000000)']['2022년06월_총인구수'].item().replace(',', ''))
        else:
            print(df_res['2022년06월_총인구수'].map(lambda x: int(x.replace(',', ''))).max())
            

In [ ]:
def find_population(sido, sigungu):
    if sido == '충북':
        sido = '충청북도'
    elif sido == '충남':
        sido = '충청남도'
    elif sido == '경북':
        sido = '경상북도'
    elif sido == '경남':
        sido = '경상남도'
    elif sido == '전북':
        sido = '전라북도'
    elif sido == '전남':
         sido = '전라남도'
    condit1 = df_sigungu_population_raw['행정구역'].str.contains(sido)
    condit2 = df_sigungu_population_raw['행정구역'].str.contains(sigungu)
    df_res = df_sigungu_population_raw[condit1 & condit2]

    if df_res.shape[0] == 0:
        return None
    elif df_res.shape[0] == 1:
        return int(df_res['2022년06월_총인구수'].item().replace(',', ''))
    else:
        res = df_res['2022년06월_총인구수'].map(lambda x: int(x.replace(',', ''))).max()
        if sido == '인천' and sigungu == '동구':
            res = df_res[df_res['행정구역'] == '인천광역시 동구 (2814000000)']['2022년06월_총인구수'].item().replace(',', '')
        if sido == '경기' and sigungu == '양주':
            res = df_res[df_res['행정구역'] == '경기도 양주시 (4163000000)']['2022년06월_총인구수'].item().replace(',', '')
        if sido == '부산' and sigungu == '서구':
            res = df_res[df_res['행정구역'] == '부산광역시 서구 (2614000000)']['2022년06월_총인구수'].item().replace(',', '')
        return int(res)

In [ ]:
df_sigungu_population['population'] = df_sigungu_population.apply(lambda x: find_population(x['sido'], x['sigungu']), axis=1)

In [ ]:
df_sigungu_population = df_sigungu_population[~df_sigungu_population['population'].isnull()].reset_index(drop=True)

### 시도시군구 인구수 구함

In [ ]:
df_sigungu_population.sort_values(['sido', 'sigungu'])

In [ ]:
def find_sigungu_confirmed_ratio(sido, sigungu, confirmed_total):
    sido_condit = df_sigungu_population['sido'] == sido 
    sigungu_condit = df_sigungu_population['sigungu'] == sigungu
    df_popul = df_sigungu_population[sido_condit & sigungu_condit]
    if df_popul.shape[0] == 0: return None
    return float(confirmed_total) / df_popul['population'].item()

### 우리 지역별 확진자수 디비에서 가져와서 csv파일에 저장한거 불러옴

In [ ]:
df_confirmed_sigungu_old = pd.read_csv('sigungu_confirmed_num.csv')

In [ ]:
df_confirmed_sigungu = pd.read_sql('SELECT * FROM shine2_covid_sigungu', kt_engine)

In [ ]:
def revert_to_int(s):
    try:
        if len(s) >=4 and s[1] == ',':
            return int(s[:1] + s[2:])
        if s[-1] == '+':
            return int(s[:-2])
        return int(s)
    except ValueError:
        print(s)
        return s

In [ ]:
df_confirmed_sigungu['confirmed_total'] = df_confirmed_sigungu['confirmed_total'].apply(revert_to_int)

In [ ]:
def get_confirmed_ratio_sigungu(standard_date, sido, sigungu, confirmed_total):
    yesterdate = (datetime.strptime(standard_date, '%Y-%m-%d') - timedelta(days=1)).strftime('%Y-%m-%d')
    condit0 = df_confirmed_sigungu['standard_date'] == standard_date
    condit1 = df_confirmed_sigungu['standard_date'] == yesterdate
    condit2 = df_confirmed_sigungu['sido'] == sido
    condit3 = df_confirmed_sigungu['sigungu'] == sigungu
    df_yesterday = df_confirmed_sigungu[(condit1 & condit2 & condit3)]
    df_today = df_confirmed_sigungu[(condit0 & condit2 & condit3)]
    if df_yesterday.shape[0] == 0: return 0
#     display(df_today)
#     display(df_yesterday)
    return confirmed_total - df_yesterday['confirmed_total'].item()

In [ ]:
# df_confirmed_sigungu['sigungu_confirmed_num'] = df_confirmed_sigungu.apply(lambda df: get_confirmed_ratio_sigungu(df['standard_date'], df['sido'], df['sigungu'], df['confirmed_total']), axis=1)

### 기존에 시군구 다 존재하던 애들에 sigungu_confirmed_ratio 붙여줌

In [ ]:
df_confirmed_sigungu['sigungu_confirmed_ratio'] = df_confirmed_sigungu.apply(lambda x : find_sigungu_confirmed_ratio(x['sido'], x['sigungu'], x['confirmed_total']), axis=1)

In [ ]:
df_confirmed_sigungu = df_confirmed_sigungu[~df_confirmed_sigungu['sigungu_confirmed_ratio'].isnull()]

In [ ]:
df_confirmed_sigungu = df_confirmed_sigungu.rename(columns={'standard_date': 'pcr_date'})

In [ ]:
df_confirmed_sigungu['pcr_date'] = df_confirmed_sigungu['pcr_date'].astype(str).apply(lambda x : x[:10])

### 전세계 유행도, 한국 유행도 붙임

In [ ]:
df_covid = pd.merge(df_covid, kor_confirmed_df, how='left', left_on='pcr_date', right_on='confirmed_date')

In [ ]:
df_covid = pd.merge(df_covid, global_confirmed_df, how='left', left_on='pcr_date', right_on='global_confirmed_date')

In [ ]:
df_covid.head()

### 시군구 유행도 붙임

In [ ]:
df_confirmed_sigungu

In [ ]:
df_covid = pd.merge(df_covid, df_confirmed_sigungu, on=['pcr_date', 'sido', 'sigungu'], how='left')

In [ ]:
display(df_confirmed_sigungu)
display(df_covid)

In [ ]:
df_covid[df_covid['sigungu_confirmed_ratio'].isnull()].shape, df_covid[~df_covid['sigungu_confirmed_ratio'].isnull()].shape

In [ ]:
df_covid['sigungu_confirmed_ratio'].unique()

In [ ]:
CONFIRMED_MAX = 11934 / 9.217 * 51.78  # 66033
GLOBAL_CONFIRMED_MAX = 1498041

In [ ]:
df_covid['confirmed_ratio'] = df_covid['confirmed_num'] / CONFIRMED_MAX
df_covid['global_confirmed_ratio'] = df_covid['global_confirmed_num'] / GLOBAL_CONFIRMED_MAX
df_covid['confirmed_week_ratio'] = df_covid['confirmed_week_num'] / CONFIRMED_MAX
df_covid['global_confirmed_week_ratio'] = df_covid['global_confirmed_week_num'] / GLOBAL_CONFIRMED_MAX

In [ ]:
df_covid = df_covid[~df_covid['confirmed_date'].isna()]

### 시군구 있는 데이터 없는 데이터 구분하는 마스킹 컬럼 추가

In [ ]:
def mask_filter(sido, sigungu):
    if sido == '' or sigungu == '': return 0
    return 1

In [ ]:
df_covid['mask'] = df_covid.apply(lambda x: mask_filter(x['sido'], x['sigungu']), axis=1)

In [ ]:
df_covid['sigungu_confirmed_ratio'] = df_covid['sigungu_confirmed_ratio'].fillna(0)

In [ ]:
israel_symptom_input_cols = ['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache']

symptom_input_cols = (['runny_nose', 'muscle_pain', 'chills',  'loss_of_taste', 
                       'loss_of_smell', 'sputum', 'chest_pain'])

surveilance_cols = (['global_confirmed_ratio', 'confirmed_ratio', 'sigungu_confirmed_ratio'])
# surveilance_cols = (['global_confirmed_ratio', 'confirmed_ratio'])

# shap value 찍어보자!! kernel explain

mask_col = ['mask']

indication_cols = ['indication_other','indication_abroad', 'indication_contact']
other_cols = ['gender', 'age_ratio', 'weekday']
res_col = ['pcr_result']

cols = (israel_symptom_input_cols + symptom_input_cols + 
        indication_cols + surveilance_cols + mask_col + 
        other_cols + res_col)

# cols = israel_symptom_input_cols + symptom_input_cols + surveilance_cols + mask_col + res_col


# symptom_input_cols = ['cough', 'fever', 'confirmed_ratio', \
#                       'sigungu_confirmed_ratio', 'mask', 'pcr_result']
#
# 'indication_other','indication_abroad', 'indication_contact'
#'mask', 'confirmed_ratio', 'sigungu_confirmed_ratio'
# 'confirmed_week_ratio', 'global_confirmed_week_ratio', 

# 0~10 기존 / 11~17 추가된 7 feature

# input_cols = ['cough', 'fever', 'sore_throat','shortness_of_breath',\
#               'head_ache', 'gender', 'runny_nose', 'muscle_pain', 'chills',\
#               'loss_of_taste', 'loss_of_smell', 'sputum', 'chest_pain', 'age_ratio', 'weekday'
#
#               'gender', 'confirmed_ratio',\
#               'global_confirmed_ratio', 'indication_other',\
#               'indication_abroad', 'indication_contact', 'pcr_result']

In [ ]:
input_cols = ['selfcheck_date'] + cols

In [ ]:
# AUC 0.91 나왔던 FEATURE라서 해본건데 그렇게 안나옴 데이터 구성에 따라 auc가 다르게 나온다

# input_cols = ['cough', 'fever', 'sore_throat','shortness_of_breath',\
#               'head_ache', 'gender', 'confirmed_ratio',\
#               'global_confirmed_ratio', 'indication_other',\
#               'indication_abroad', 'indication_contact',\
#               'runny_nose', 'muscle_pain', 'chills',\
#               'loss_of_taste', 'loss_of_smell', 'sputum', 'chest_pain', 'age_ratio',\
#               'confirmed_week_ratio', 'global_confirmed_week_ratio',\
#               'weekday','pcr_result']

In [ ]:
df_input = df_covid[~df_covid['confirmed_ratio'].isna()]

### mask를 이용해서 시군구 있는 애들만 뽑는다 (실험 조건 중 하나)

In [ ]:
# df_input = df_input[df_input['mask'] == 1]

In [ ]:
df_input['pcr_result'].unique()

In [ ]:
neg_cnt = df_input[df_input['pcr_result'] == 0].shape[0]
neg_cnt

### 양성 갯수 줄이는 작업인데 안했음

In [ ]:
# df_input_balanced = df_input.sort_values('pcr_result').iloc[:neg_cnt*2]

#### 이벤트 기간
1차: 2.28 - 4.4
2차: 8.16 - 10.31

In [ ]:
event_start_date_1 = '2022-02-14'
event_end_date_1 = '2022-04-18'
event_start_date_2 = '2022-08-02'
event_end_date_2 = '2022-11-14'

In [ ]:
# df_input_balanced = df_input[(event_start_date_2 <= df_input['test_date']) & (df_input['test_date'] <= event_end_date_2)]
df_input_balanced = df_input

In [ ]:
from collections import Counter

In [ ]:
baby_id_counter = Counter(df_input_balanced['patient_id'])

In [ ]:
baby_ids_only_one = [x for x, count in baby_id_counter.items() if count == 1]
len(baby_ids_only_one)

In [ ]:
baby_ids = df_input_balanced['patient_id'].unique()

In [ ]:
baby_ids.shape

In [ ]:
Counter(baby_id_counter.values())

In [ ]:
# df_input_balanced = df_input_balanced[df_input_balanced['baby_id'].isin(baby_ids_only_one)]

In [ ]:
# df_input_balanced

In [ ]:
# input_cols.append('baby_id')

In [ ]:
input_cols = ['patient_id'] + input_cols
input_cols

In [ ]:
df_input_final = df_input_balanced[input_cols]

In [ ]:
df_input_final.sort_values('selfcheck_date')

In [ ]:
df_input_final['global_confirmed_ratio'] = df_input_final['global_confirmed_ratio'] / df_input_final['global_confirmed_ratio'].max() 
df_input_final['confirmed_ratio'] = df_input_final['confirmed_ratio'] / df_input_final['confirmed_ratio'].max()
df_input_final['sigungu_confirmed_ratio'] = df_input_final['sigungu_confirmed_ratio'] / df_input_final['sigungu_confirmed_ratio'].max()

In [ ]:
(df_input_final['global_confirmed_ratio'].max(), 
 df_input_final['confirmed_ratio'].max(), 
 df_input_final['sigungu_confirmed_ratio'].max())

### 최종 데이터셋 음성 양성 갯수

In [ ]:
df_input_final[df_input_final['pcr_result'] == 0].shape, df_input_final[df_input_final['pcr_result'] == 1].shape

In [ ]:
def split_input_gt(non_split_data):
    splited_input = non_split_data[:, :-1]
    splited_gt = non_split_data[:, -1]
    
    return splited_input, splited_gt

In [ ]:
def split_train_test(data, test_ratio):
#     shuffled_indices = np.random.permutation(len(data)).astype(int)
    test_set_size = int(len(data) * test_ratio)
    
#     test_indices = shuffled_indices[:test_set_size]
#     train_indices = shuffled_indices[test_set_size:]
    
    print(len(data))
#     print(test_indices.dtype)
#     print(train_indices.dtype)
    
    test_data = data[:test_set_size]
    train_data = data[test_set_size:]
#     test_data = data[test_indices]
#     train_data = data[train_indices]
    
    return train_data, test_data

In [ ]:
def split_df_with_ratio(df, ratio=0.2):
    baby_ids = df['patient_id'].unique()
    np.random.shuffle(baby_ids)
    test_set_size = int(len(baby_ids) * ratio)
    test_ids = baby_ids[:test_set_size]
    train_ids = baby_ids[test_set_size:]
    print(f'baby_ids가 나눠진 갯수 {len(train_ids)} {len(test_ids)}')
    return df[df['patient_id'].isin(train_ids)], df[df['patient_id'].isin(test_ids)]

In [ ]:
# input_arr = df_input_final.values

In [ ]:
# train_datas, test_datas = split_train_test(input_arr, test_ratio=0.4)

# train_input, train_gt = split_input_gt(train_datas)
# test_input, test_gt = split_input_gt(test_datas)

In [ ]:
for col in input_cols:
    if df_input_final[df_input_final[col].isna()].shape[0] != 0:
        df_input_final[df_input_final[col].isna()] = df_input_final[df_input_final[col].isna()].fillna(0)

In [ ]:
np.random.seed(1212)

In [ ]:
df_train, df_test = split_df_with_ratio(df_input_final, 0.4)
df_test, df_vaild = split_df_with_ratio(df_test, 0.5)

In [ ]:
df_train['pcr_result'].value_counts()

In [ ]:
df_test['pcr_result'].value_counts()

In [ ]:
df_vaild['pcr_result'].value_counts()

In [ ]:
df_train.shape, df_test.shape, df_vaild.shape

In [ ]:
# df_train.to_csv('df_train_v3_1.csv', index=False)
# df_vaild.to_csv('df_valid_v3_1.csv', index=False)
# df_test.to_csv('df_test_v3_1.csv', index=False)

In [ ]:
# df_train_mc = pd.read_csv('df_train_mc.csv')
# df_test_mc = pd.read_csv('df_test.csv')

In [ ]:
df_train.values[:, 1:]

In [ ]:
train_datas = df_train.values[:, 2:]
test_datas = df_test.values[:, 2:]

train_input, train_gt = split_input_gt(train_datas)
test_input, test_gt = split_input_gt(test_datas)

In [ ]:
train_gt = train_gt.astype(float)
test_gt = test_gt.astype(float)

In [ ]:
train_input.shape, train_gt.shape, test_input.shape, test_gt.shape

In [ ]:
train_input = train_input.astype(float)
test_input = test_input.astype(float)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib

In [ ]:
logistic_clf = LogisticRegression(C=1000.0, random_state=0)

In [ ]:
logistic_clf.fit(train_input, train_gt)

In [ ]:
train_input

In [ ]:
# explainer = shap.Explainer(logistic_clf)
input_cols[1:-1]

In [ ]:
explainer = shap.Explainer(logistic_clf, train_input, feature_names=input_cols[1:-1])
shap_values = explainer(test_input)

In [ ]:
shap.plots.beeswarm(shap_values)

In [ ]:
shap.plots.bar(shap_values)

In [ ]:
shap.initjs()
shap.plots.force(shap_values[0])

In [ ]:
# shap.plots.force(shap_values)

In [ ]:
pred = logistic_clf.predict(test_input)

In [ ]:
accuracy_score(test_gt, pred)

In [ ]:
display_performance(logistic_clf, pred, test_input, test_gt)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(in_features=16, out_features=1)
        self.activation = nn.Sigmoid()
        
    def forward(self, x):
        x = self.fc(x)
        x = self.activation(x)
        x = x.reshape(-1, 1)
        return x


In [ ]:
net = Net()
print(net)
print(net.fc.weight, net.fc.bias)

device = 'cpu'
optimizer = optim.Adam(net.parameters(), lr=0.1)
criterion = nn.BCELoss()

In [ ]:
class ShineDataset(Dataset):
    def __init__(self, input_data, gt, transform=None, target_transform=None):
        self.input_data = input_data
        self.gt = gt
        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return len(self.input_data)
    
    def __getitem__(self, idx):
        return self.input_data[idx], self.gt[idx]

In [ ]:
def plot_roc_curve_for_dnn(fper, tper):
    plt.plot(fper, tper, color='red', label='ROC')
    plt.plot([0, 1], [0, 1], color='green', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic Curve')
    plt.legend()
    plt.show()

In [ ]:
training_data = ShineDataset(torch.Tensor(train_input), torch.Tensor(train_gt.reshape(-1, 1)))
valid_data = ShineDataset(torch.Tensor(test_input), torch.Tensor(test_gt.reshape(-1, 1)))

train_dataloader = DataLoader(training_data, batch_size = 128, shuffle=False)
valid_dataloader = DataLoader(valid_data, batch_size=64, shuffle=False)

In [ ]:
train_input.shape